# Импорты

In [ ]:
!pip install qiskit ipywidgets

In [ ]:
import numpy as np
import psycopg2
from qiskit import Aer
import qiskit
from qiskit import *
from qiskit import QuantumCircuit, transpile, execute
from qiskit.providers.aer import QasmSimulator
from qiskit.visualization import array_to_latex
from qiskit.quantum_info import Statevector
from qiskit.visualization import plot_histogram, plot_state_city
from qiskit import transpile
from qiskit_aer import AerSimulator
from qiskit.circuit.library.standard_gates import XGate
from qiskit.circuit.library.standard_gates import RYGate
import math as math
import sys as sys
import itertools
from qiskit.visualization import plot_histogram

# Абстракции

In [ ]:
get_bin = lambda x, n: format(x, 'b').zfill(n)
get_bin_len = lambda x: len(format(x, 'b'))

def get_combinations_bitwise(array):
    sz = len(array)
    all_combinations = []
    for mask in range ((1<<sz)):
        comb = []
        for pos in range(sz):
            if (mask & (1 << pos)):
                comb.append(array[pos])
        all_combinations.append(comb)
    return list(map(lambda x: sum(x), all_combinations))

In [ ]:
class CoefficientGenerator():
    def __init__(self, n, t_log) -> None:
        self.n = n
        self.m = int(math.pow(2, self.n)) - 1
        self.m_f = self.m + 1
        self.t_log = t_log
        self.t = int(math.pow(2, t_log))
        self.e = 2*math.log(2*self.m_f)/self.t
    
    def calculate_formula(self, k_array, b):
        return (1/math.pow(self.t, 2))*(math.pow(sum(math.cos((2*math.pi*int(k_i)*b)/self.m_f) for k_i in k_array), 2))

    def analize_k(self, k_array):
        return max(map(lambda b_i: self.calculate_formula(k_array, b_i),  range(1, self.m)))

    def is_k_good(self, k_array):
        return self.analize_k(k_array) < self.e
    
    def generate_good_k(self):
        k = np.random.choice(list(range(1, self.m_f)), size=self.t)
        while(self.is_k_good(k) == False):
            k = np.random.choice(list(range(1, self.m_f)), size=self.t)

        k_score = self.analize_k(k)
        for i in range(1000):
            test_k = np.random.choice(list(range(1, self.m_f)), size=self.t)
            test_k_score = self.analize_k(test_k)
            if(test_k_score < k_score):
                k_score = test_k_score
                k = test_k

        return k

    def is_d_good(self, d):
        return self.is_k_good(get_combinations_bitwise(d))

    def analize_d(self, d):
        return self.analize_k(get_combinations_bitwise(d))
    
    def generate_good_d(self):
        d = np.random.choice(list(range(1, self.m_f)), size=self.t_log)
        while(self.is_d_good(d) == False):
            d = np.random.choice(list(range(1, self.m_f)), size=self.t_log)

        d_score = self.analize_d(d)
        for i in range(1000):
            d_test = np.random.choice(list(range(1, self.m_f)), size=self.t_log)
            d_test_score = self.analize_d(d_test)
            if(d_test_score < d_score):
                d = d_test
                d_score = d_test_score

        return d

In [ ]:
class AnglesGenerator():
    def __init__(self, number, t_log) -> None:
        self.number = number
        self.t_log = t_log
        self.n = get_bin_len(number)
        self.m = int(math.pow(2, self.n)) - 1
        self.m_f = self.m + 1
    
    def __get_angle(self, k_i):
        return (4*math.pi*k_i*self.number)/self.m_f
    
    def get_angles(self, coefficients):
        return list(map(lambda x: self.__get_angle(x), coefficients))

In [ ]:
class Schema:
    def __init__(self, qubits_number) -> None:
        self.qubits_number = qubits_number
        self.registers = QuantumRegister(qubits_number)
        self.circuit = QuantumCircuit(self.registers)
        self.t_log = self.qubits_number - 1
    
    def rotateY_last_qubit_full_control(self, angles, angle_multiplier = 1):
        for i in range(len(angles)):
            gate = RYGate(angle_multiplier * angles[i])
            self.circuit.append(gate.control(self.t_log, ctrl_state=get_bin(i, self.t_log)), self.registers)
    
    def rotateY_last_qubit_one_qbit_control(self, angles):
        for i in range(len(angles)):
            gate = RYGate(angles[i])
            self.circuit.append(gate.control(1), [i, self.t_log])
    
    def h_all_but_last_qubits(self):
        for i in range(self.qubits_number-1):
            self.circuit.h(i)
    
    def draw(self):
        return self.circuit.draw()
    
    def measure(self):
        self.circuit.measure_all()
        simulator = Aer.get_backend('qasm_simulator')
        job = execute(self.circuit, simulator, shots=100)
        result = job.result()
        counts = result.get_counts(self.circuit)
        return plot_histogram(counts)
    
    def get_state(self):
        simulator = Aer.get_backend('statevector_simulator')
        job = execute(self.circuit, simulator, shots=1)
        result = job.result()
        return job_result.get_statevector(self.circuit)

### Генерация обычной схемы

In [ ]:
usual_depth_coefficient_generator = CoefficientGenerator(get_bin_len(245), 4)
usual_depth_angles_generator = AnglesGenerator(245, 4)
usual_depth_schema = Schema(5)

usual_depth_schema.h_all_but_last_qubits()
usual_depth_schema.rotateY_last_qubit_full_control(usual_depth_angles_generator.get_angles(usual_depth_coefficient_generator.generate_good_k()))
usual_depth_schema.draw()

In [ ]:
usual_depth_schema.measure()

### Генерация схемы малой глубины

In [ ]:
short_depth_coefficient_generator = CoefficientGenerator(get_bin_len(245), 4)
short_depth_angles_generator = AnglesGenerator(245, 4)
short_depth_schema = Schema(5)

short_depth_schema.h_all_but_last_qubits()
short_depth_schema.rotateY_last_qubit_one_qbit_control(short_depth_angles_generator.get_angles(short_depth_coefficient_generator.generate_good_d()))
short_depth_schema.draw()

In [ ]:
short_depth_schema.measure()

### Проверка работоспособности

In [ ]:
short_depth_coefficient_generator = CoefficientGenerator(get_bin_len(245), 4)
short_depth_angles_generator = AnglesGenerator(245, 4)
short_depth_schema = Schema(5)
good_d = short_depth_coefficient_generator.generate_good_d()

short_depth_schema.h_all_but_last_qubits()
short_depth_schema.rotateY_last_qubit_one_qbit_control(usual_depth_angles_generator.get_angles(good_d))
short_depth_schema.rotateY_last_qubit_full_control(usual_depth_angles_generator.get_angles(get_combinations_bitwise(good_d)), angle_multiplier=-1)
short_depth_schema.h_all_but_last_qubits()
short_depth_schema.draw()

In [ ]:
short_depth_schema.measure()

### Сравнение алгоритмов

In [ ]:
def compare(input):
    generator = CoefficientGenerator(input[0], input[1])
    k = generator.generate_good_k()
    d = generator.generate_good_d()
    result = [generator.analize_k(k), generator.analize_d(d)]
    return result

In [ ]:
tests = []
for i in range(1,11):
    for j in range(1, i):
        tests.append([i,j])

tests

In [ ]:
for test in tests:
    result = compare(test)
    print(f'{test}: {result}')

### Генерация всех схем

**Результаты прдставлены в этом репозитории**

In [ ]:
conn = psycopg2.connect(
            host="localhost",
            database="postgres",
            user="postgres",
            password="postgres")
cursor = conn.cursor()

tests = [[5,3], [6, 4]]

for test in tests:
    generator = CoefficientGenerator(test[0], test[1])
    all_d = list(itertools.combinations_with_replacement(range(1, generator.m_f), test[1]))
    print(len(all_d))

    for d_s in all_d:
        d_k = generator.analize_d(d_s)
        cursor.execute(f"""INSERT INTO d_test_results (bits_number, qubits_number, coefficients, result) VALUES ({test[0]}, {test[1]}, '{d_s}', {d_k})""")
        conn.commit()

cursor.close()
conn.close()

In [ ]:
conn = psycopg2.connect(
            host="localhost",
            database="postgres",
            user="postgres",
            password="postgres")
cursor = conn.cursor()

tests = [[5,3]]

for test in tests:
    generator = CoefficientGenerator(test[0], test[1])
    all_k = list(itertools.combinations_with_replacement(range(1, generator.m_f), generator.t))
    print(len(all_k))

    for k_s in all_k:
        k_k = generator.analize_k(k_s)
        cursor.execute(f"""INSERT INTO k_test_results (bits_number, qubits_number, coefficients, result) VALUES ({test[0]}, {test[1]}, '{k_s}', {k_k})""")
        conn.commit()

cursor.close()
conn.close()